# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f028cd92d90>
├── 'a' --> tensor([[-0.6093, -0.9796, -1.6916],
│                   [ 0.6389, -0.0124,  1.9296]])
└── 'x' --> <FastTreeValue 0x7f028cd494f0>
    └── 'c' --> tensor([[ 0.8127,  0.0227, -0.1015,  0.7801],
                        [-0.2804, -1.1122,  1.0922, -0.2541],
                        [ 1.0530, -0.4106, -0.6510, -0.5090]])

In [4]:
t.a

tensor([[-0.6093, -0.9796, -1.6916],
        [ 0.6389, -0.0124,  1.9296]])

In [5]:
%timeit t.a

76.2 ns ± 0.777 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f028cd92d90>
├── 'a' --> tensor([[1.2914, 0.2086, 0.0058],
│                   [0.4309, 1.2050, 0.5533]])
└── 'x' --> <FastTreeValue 0x7f028cd494f0>
    └── 'c' --> tensor([[ 0.8127,  0.0227, -0.1015,  0.7801],
                        [-0.2804, -1.1122,  1.0922, -0.2541],
                        [ 1.0530, -0.4106, -0.6510, -0.5090]])

In [7]:
%timeit t.a = new_value

81.7 ns ± 0.438 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.6093, -0.9796, -1.6916],
               [ 0.6389, -0.0124,  1.9296]]),
    x: Batch(
           c: tensor([[ 0.8127,  0.0227, -0.1015,  0.7801],
                      [-0.2804, -1.1122,  1.0922, -0.2541],
                      [ 1.0530, -0.4106, -0.6510, -0.5090]]),
       ),
)

In [10]:
b.a

tensor([[-0.6093, -0.9796, -1.6916],
        [ 0.6389, -0.0124,  1.9296]])

In [11]:
%timeit b.a

74 ns ± 0.574 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.7134, -0.0742,  0.1632],
               [ 0.0464,  0.7477, -1.3023]]),
    x: Batch(
           c: tensor([[ 0.8127,  0.0227, -0.1015,  0.7801],
                      [-0.2804, -1.1122,  1.0922, -0.2541],
                      [ 1.0530, -0.4106, -0.6510, -0.5090]]),
       ),
)

In [13]:
%timeit b.a = new_value

642 ns ± 6.31 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.14 µs ± 15.1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.6 µs ± 214 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

283 µs ± 7.07 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

259 µs ± 9.49 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f01b26a57f0>
├── 'a' --> tensor([[[-0.6093, -0.9796, -1.6916],
│                    [ 0.6389, -0.0124,  1.9296]],
│           
│                   [[-0.6093, -0.9796, -1.6916],
│                    [ 0.6389, -0.0124,  1.9296]],
│           
│                   [[-0.6093, -0.9796, -1.6916],
│                    [ 0.6389, -0.0124,  1.9296]],
│           
│                   [[-0.6093, -0.9796, -1.6916],
│                    [ 0.6389, -0.0124,  1.9296]],
│           
│                   [[-0.6093, -0.9796, -1.6916],
│                    [ 0.6389, -0.0124,  1.9296]],
│           
│                   [[-0.6093, -0.9796, -1.6916],
│                    [ 0.6389, -0.0124,  1.9296]],
│           
│                   [[-0.6093, -0.9796, -1.6916],
│                    [ 0.6389, -0.0124,  1.9296]],
│           
│                   [[-0.6093, -0.9796, -1.6916],
│                    [ 0.6389, -0.0124,  1.9296]]])
└── 'x' --> <FastTreeValue 0x7f01ba531c10>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

46.3 µs ± 875 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f01b25d6c10>
├── 'a' --> tensor([[-0.6093, -0.9796, -1.6916],
│                   [ 0.6389, -0.0124,  1.9296],
│                   [-0.6093, -0.9796, -1.6916],
│                   [ 0.6389, -0.0124,  1.9296],
│                   [-0.6093, -0.9796, -1.6916],
│                   [ 0.6389, -0.0124,  1.9296],
│                   [-0.6093, -0.9796, -1.6916],
│                   [ 0.6389, -0.0124,  1.9296],
│                   [-0.6093, -0.9796, -1.6916],
│                   [ 0.6389, -0.0124,  1.9296],
│                   [-0.6093, -0.9796, -1.6916],
│                   [ 0.6389, -0.0124,  1.9296],
│                   [-0.6093, -0.9796, -1.6916],
│                   [ 0.6389, -0.0124,  1.9296],
│                   [-0.6093, -0.9796, -1.6916],
│                   [ 0.6389, -0.0124,  1.9296]])
└── 'x' --> <FastTreeValue 0x7f028cd92c70>
    └── 'c' --> tensor([[ 0.8127,  0.0227, -0.1015,  0.7801],
                        [-0.2804, -1.1122,  1.0922, -0.2541],
                 

In [23]:
%timeit t_cat(trees)

41.9 µs ± 1.01 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

94.1 µs ± 2.09 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.6093, -0.9796, -1.6916],
                [ 0.6389, -0.0124,  1.9296]],
       
               [[-0.6093, -0.9796, -1.6916],
                [ 0.6389, -0.0124,  1.9296]],
       
               [[-0.6093, -0.9796, -1.6916],
                [ 0.6389, -0.0124,  1.9296]],
       
               [[-0.6093, -0.9796, -1.6916],
                [ 0.6389, -0.0124,  1.9296]],
       
               [[-0.6093, -0.9796, -1.6916],
                [ 0.6389, -0.0124,  1.9296]],
       
               [[-0.6093, -0.9796, -1.6916],
                [ 0.6389, -0.0124,  1.9296]],
       
               [[-0.6093, -0.9796, -1.6916],
                [ 0.6389, -0.0124,  1.9296]],
       
               [[-0.6093, -0.9796, -1.6916],
                [ 0.6389, -0.0124,  1.9296]]]),
    x: Batch(
           c: tensor([[[ 0.8127,  0.0227, -0.1015,  0.7801],
                       [-0.2804, -1.1122,  1.0922, -0.2541],
                       [ 1.0530, -0.4106, -0.6510, -0.5090]],
         

In [26]:
%timeit Batch.stack(batches)

109 µs ± 792 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.6093, -0.9796, -1.6916],
               [ 0.6389, -0.0124,  1.9296],
               [-0.6093, -0.9796, -1.6916],
               [ 0.6389, -0.0124,  1.9296],
               [-0.6093, -0.9796, -1.6916],
               [ 0.6389, -0.0124,  1.9296],
               [-0.6093, -0.9796, -1.6916],
               [ 0.6389, -0.0124,  1.9296],
               [-0.6093, -0.9796, -1.6916],
               [ 0.6389, -0.0124,  1.9296],
               [-0.6093, -0.9796, -1.6916],
               [ 0.6389, -0.0124,  1.9296],
               [-0.6093, -0.9796, -1.6916],
               [ 0.6389, -0.0124,  1.9296],
               [-0.6093, -0.9796, -1.6916],
               [ 0.6389, -0.0124,  1.9296]]),
    x: Batch(
           c: tensor([[ 0.8127,  0.0227, -0.1015,  0.7801],
                      [-0.2804, -1.1122,  1.0922, -0.2541],
                      [ 1.0530, -0.4106, -0.6510, -0.5090],
                      [ 0.8127,  0.0227, -0.1015,  0.7801],
                      [-0.2804, -

In [28]:
%timeit Batch.cat(batches)

199 µs ± 9.02 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

598 µs ± 20.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
